In [1]:
#import statements, hyper params, models
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import copy
from matplotlib.pyplot import figure

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings('ignore') 
from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from numpy import random
from random import randrange
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import math
import time
    
THRESH_HOLD=5e-6            #MSE value when we consider the second model to have "cracked" the first
EPOCHS = 25                 #number of passes of whole data
BATCH_SIZE = 64             #size of data going through at once
LEARNING_RATE = 0.01       #how much we shift parameters during back prop
NEURONS=5                   #number of nodes in each layer
XSIZE=1056                  #size of each data set generated
BADLOSSPASSES=1000          #number of times we have small loss until we stop code
TESTMODULE_LAYERS=10        #number of layers in cracking module
GROUND_TRUTH_LAYERS=5       #number of layers in ground thruth models

    
    
def get_Xvals():
    return (np.random.uniform( size=(XSIZE, 5))-0.5)*np.sqrt(12)

    
    
    
def getdata_XY():
    X=(np.random.uniform( size=(XSIZE, 5))-0.5)*np.sqrt(12)
    test_data = testData(torch.FloatTensor(X))
    test_loader = DataLoader(dataset=test_data, batch_size=1)
    y_truthlist =[]
    
    
    GroundTruth.eval()
    with torch.no_grad():
        for X_batch in test_loader:
            X_batch = X_batch.to(device)
            y_test_pred = GroundTruth(X_batch)
            y_test_pred = torch.sigmoid(y_test_pred)
            y_truthlist.append(y_test_pred.cpu().numpy())


    y_truthlist = [a.squeeze().tolist() for a in y_truthlist]
    return X, y_truthlist

def get_yvals(model, X_data):
    model.eval()
    test_data = testData(torch.FloatTensor(X_data))
    test_loader = DataLoader(dataset=test_data, batch_size=1)
    y_truthlist=[]
    with torch.no_grad():
        for X_batch in test_loader:
            X_batch = X_batch.to(device)
            y_test_pred = model(X_batch)
            y_test_pred = torch.sigmoid(y_test_pred)
            y_truthlist.append(y_test_pred.cpu().numpy())


    y_truthlist = [a.squeeze().tolist() for a in y_truthlist]
    return y_truthlist



#Data class
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)

#Data loaders (to look at data) and deal with data (training and testing)
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    
def binary_acc(y_pred, y_test): 
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

def trainnetwork(network, train_loader,Time=True):
    t0=time.time()
    network.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(network.parameters(), lr=LEARNING_RATE)


    network.train()
    for e in range(1, EPOCHS+1):
        epoch_loss = 0
        epoch_acc = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()

            #X_batch = X_batch.astype(np.float32)
            
            
            
            
            y_pred = torch.sigmoid(network(X_batch))
            #y_pred = network(X_batch)

            loss = criterion(y_pred, y_batch.unsqueeze(1))
            acc = binary_acc(y_pred, y_batch.unsqueeze(1))

            loss.backward()
            optimizer.step()
            #print("batch loss: ", loss.item()) 
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            #print("batch loss: ", loss.item()) 

        #print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f}')
        #rint("Ave Loss: ", epoch_loss)
        #rint("size ", len(train_loader))
        
            
    t1=time.time()
    return(t1-t0)



def get_new_GroundTruth(model_list, time_list, time):
    better_model_list=[]
    for (i, model) in zip(time_list, model_list):
        if i>time:
            better_model_list.append(model)
    
    
    newGroundTruth=GroundTruth.get_copy()
    
    
    layer1_weight=torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
    for i in better_model_list:
        layer1_weight=layer1_weight+i.layer_1.weight/(len(better_model_list))
        
    layer1_weight.requires_grad_()
    newGroundTruth.layer_1.weight=torch.nn.Parameter(layer1_weight)

    layer2_weight=torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
    for i in better_model_list:
        layer2_weight=layer2_weight+i.layer_2.weight/(len(better_model_list))
        
    layer2_weight.requires_grad_()
    newGroundTruth.layer_2.weight=torch.nn.Parameter(layer2_weight)
    
    
    layer3_weight=torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
    for i in better_model_list:
        layer3_weight=layer3_weight+i.layer_3.weight/(len(better_model_list))
        
    layer3_weight.requires_grad_()
    newGroundTruth.layer_3.weight=torch.nn.Parameter(layer3_weight)
    
    layer4_weight=torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
    for i in better_model_list:
        layer4_weight=layer4_weight+i.layer_4.weight/(len(better_model_list))
        
    layer4_weight.requires_grad_()
    newGroundTruth.layer_4.weight=torch.nn.Parameter(layer4_weight)
    
    layer5_weight=torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
    for i in better_model_list:
        layer5_weight=layer5_weight+i.layer_5.weight/(len(better_model_list))
        
    layer5_weight.requires_grad_()
    newGroundTruth.layer_5.weight=torch.nn.Parameter(layer5_weight)
    
                
    layeroutput_weight=torch.tensor([[0,0,0,0,0]],requires_grad=False)
    for i in better_model_list:
        layeroutput_weight=layeroutput_weight+i.layer_out.weight/(len(better_model_list))
        
    layeroutput_weight.requires_grad_()
    newGroundTruth.layer_out.weight=torch.nn.Parameter(layeroutput_weight)
    
    return newGroundTruth
        
def check_threshhold(network, test_loader, truth):
    total_loss=0
    total_lossconfirm=0
    #criterion = nn.MSELoss()
    network.eval()
    with torch.no_grad():
        for (X_batch, y) in zip(test_loader, truth):
            X_batch = X_batch.to(device)
            y_test_pred = network(X_batch)
            y_pred_tag = torch.sigmoid(y_test_pred)
            #loss = criterion(y_pred_tag, torch.tensor([y]))

            
            #print("y ", y)
            #print("pred ", y_pred_tag.cpu().numpy())
            total_loss=total_loss+((y_pred_tag.cpu().numpy()-y))**2
            #total_lossconfirm=total_lossconfirm+loss.item()
    
    #total_loss=np.sqrt(total_loss)/len(truth)
    total_loss=total_loss/len(truth)
    total_lossconfirm=total_lossconfirm/len(truth)
    #print("current loss: ", total_loss)
    #print("current confirm loss: ", total_lossconfirm)

    if total_loss[0][0]<THRESH_HOLD:
        return True, total_loss[0][0]
    else:
        return False, total_loss[0][0]
        
        
def evalnetwork(network, test_loader, stats=True):
    y_pred_list =[]
    network.eval()
    with torch.no_grad():
        for X_batch in test_loader:
            X_batch = X_batch.to(device)
            
            y_test_pred = network(X_batch)
            y_test_pred = torch.sigmoid(y_test_pred)
            #y_pred_tag = torch.round(y_test_pred)
            y_pred_list.append(y_test_pred.cpu().numpy())


    y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
    if(stats):
        statistics(y_pred_list)
    return y_pred_list

def statistics(y_pred_list):
    correctcounter=0
    for i in range(len(y_pred_list)):
        if y_pred_list[i]==y_test[i]:
            correctcounter=correctcounter+1       

    print("# correct: ",correctcounter, " out of ", len(y_test))

    print("Model statistics")
    print(classification_report(y_test, y_pred_list))    
    
#model class 5 layers, then an output layer
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        self.layer_1 = nn.Linear(5, NEURONS) 
        self.layer_2 = nn.Linear(NEURONS, NEURONS) 
        self.layer_3 = nn.Linear(NEURONS, NEURONS)
        self.layer_4 = nn.Linear(NEURONS,NEURONS)
        self.layer_5 = nn.Linear(NEURONS, NEURONS) 
        self.layer_out = nn.Linear(NEURONS, 1) #output layer
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        
        self.batchnorm1 = nn.BatchNorm1d(NEURONS)
        self.batchnorm2 = nn.BatchNorm1d(NEURONS)
        self.batchnorm3 = nn.BatchNorm1d(NEURONS)
        self.batchnorm4 = nn.BatchNorm1d(NEURONS)
        self.batchnorm5 = nn.BatchNorm1d(NEURONS)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.relu(self.layer_3(x))
        x = self.batchnorm3(x)
        x = self.relu(self.layer_4(x))
        x = self.batchnorm4(x)
        x = self.relu(self.layer_5(x))
        x = self.batchnorm5(x)        
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x
     
    def get_copy(self):
        copied_model=binaryClassification()
        copied_model.layer_1 = copy.deepcopy(self.layer_1)
        copied_model.layer_2 = copy.deepcopy(self.layer_2) 
        copied_model.layer_3 = copy.deepcopy(self.layer_3)
        copied_model.layer_4 = copy.deepcopy(self.layer_4)
        copied_model.layer_5 = copy.deepcopy(self.layer_5)
        copied_model.layer_out = copy.deepcopy(self.layer_out)
        copied_model.batchnorm1=copy.deepcopy(self.batchnorm1)
        copied_model.batchnorm2=copy.deepcopy(self.batchnorm2)
        copied_model.batchnorm3=copy.deepcopy(self.batchnorm3)
        copied_model.batchnorm4=copy.deepcopy(self.batchnorm4)
        copied_model.batchnorm5=copy.deepcopy(self.batchnorm5)

        
          
        return copied_model
  
    
    def change_node_model(self, node):
        changed_node=binaryClassification()
        changed_node.layer_1 = copy.deepcopy(self.layer_1)
        changed_node.layer_2 = copy.deepcopy(self.layer_2) 
        changed_node.layer_3 = copy.deepcopy(self.layer_3)
        changed_node.layer_4 = copy.deepcopy(self.layer_4)
        changed_node.layer_5 = copy.deepcopy(self.layer_5)
        changed_node.layer_out = copy.deepcopy(self.layer_out) 
        
        weightchange=np.random.normal(0, 1, size=5)
        weightchange=[i.item() for i in weightchange]

        if node[0]==0:
            self.layer_1.weight.requires_grad_(False)
            if node[1]==0:
                newweight=self.layer_1.weight+torch.tensor([weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_1.weight=torch.nn.Parameter(newweight)
                
            if node[1]==1:
                newweight=self.layer_1.weight+torch.tensor([[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)  
                newweight.requires_grad_()
                changed_node.layer_1.weight=torch.nn.Parameter(newweight)
                
            if node[1]==2:
                newweight=self.layer_1.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)          
                newweight.requires_grad_()
                changed_node.layer_1.weight=torch.nn.Parameter(newweight)
                
            if node[1]==3:
                newweight=self.layer_1.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0]],requires_grad=False)       
                newweight.requires_grad_()
                changed_node.layer_1.weight=torch.nn.Parameter(newweight)
                
            if node[1]==4:
                newweight=self.layer_1.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange],requires_grad=False)           
                newweight.requires_grad_()
                changed_node.layer_1.weight=torch.nn.Parameter(newweight)

        elif node[0]==1:
            self.layer_2.weight.requires_grad_(False)
            if node[1]==0:
                newweight=self.layer_2.weight+torch.tensor([weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_2.weight=torch.nn.Parameter(newweight)
                
            if node[1]==1:
                newweight=self.layer_2.weight+torch.tensor([[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)  
                newweight.requires_grad_()
                changed_node.layer_2.weight=torch.nn.Parameter(newweight)
                
            if node[1]==2:
                newweight=self.layer_2.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)          
                newweight.requires_grad_()
                changed_node.layer_2.weight=torch.nn.Parameter(newweight)
                
            if node[1]==3:
                newweight=self.layer_2.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0]],requires_grad=False)       
                newweight.requires_grad_()
                changed_node.layer_2.weight=torch.nn.Parameter(newweight)
                
            if node[1]==4:
                newweight=self.layer_2.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange],requires_grad=False)           
                newweight.requires_grad_()
                changed_node.layer_2.weight=torch.nn.Parameter(newweight)
   
        elif node[0]==2:
            if node[1]==0:
                newweight=self.layer_3.weight.requires_grad_(False)+torch.tensor([weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_3.weight=torch.nn.Parameter(newweight)
                
            if node[1]==1:
                newweight=self.layer_3.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)  
                newweight.requires_grad_()
                changed_node.layer_3.weight=torch.nn.Parameter(newweight)
                
            if node[1]==2:
                newweight=self.layer_3.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)          
                newweight.requires_grad_()
                changed_node.layer_3.weight=torch.nn.Parameter(newweight)
                
            if node[1]==3:
                newweight=self.layer_3.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0]],requires_grad=False)       
                newweight.requires_grad_()
                changed_node.layer_3.weight=torch.nn.Parameter(newweight)
                
            if node[1]==4:
                newweight=self.layer_3.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange],requires_grad=False)           
                newweight.requires_grad_()
                changed_node.layer_3.weight=torch.nn.Parameter(newweight)
                               
        elif node[0]==3:
            if node[1]==0:
                newweight=self.layer_4.weight.requires_grad_(False)+torch.tensor([weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_4.weight=torch.nn.Parameter(newweight)
                
            if node[1]==1:
                newweight=self.layer_4.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)  
                newweight.requires_grad_()
                changed_node.layer_4.weight=torch.nn.Parameter(newweight)
                
            if node[1]==2:
                newweight=self.layer_4.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)          
                newweight.requires_grad_()
                changed_node.layer_4.weight=torch.nn.Parameter(newweight)
                
            if node[1]==3:
                newweight=self.layer_4.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0]],requires_grad=False)       
                newweight.requires_grad_()
                changed_node.layer_4.weight=torch.nn.Parameter(newweight)
                
            if node[1]==4:
                newweight=self.layer_4.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange],requires_grad=False)           
                newweight.requires_grad_()
                changed_node.layer_4.weight=torch.nn.Parameter(newweight)
             
        elif node[0]==4:
            if node[1]==0:
                newweight=self.layer_5.weight.requires_grad_(False)+torch.tensor([weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_5.weight=torch.nn.Parameter(newweight)
                
            if node[1]==1:
                newweight=self.layer_5.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)  
                newweight.requires_grad_()
                changed_node.layer_5.weight=torch.nn.Parameter(newweight)
                
            if node[1]==2:
                newweight=self.layer_5.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)          
                newweight.requires_grad_()
                changed_node.layer_5.weight=torch.nn.Parameter(newweight)
                
            if node[1]==3:
                newweight=self.layer_5.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0]],requires_grad=False)       
                newweight.requires_grad_()
                changed_node.layer_5.weight=torch.nn.Parameter(newweight)
                
            if node[1]==4:
                newweight=self.layer_5.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange],requires_grad=False)           
                newweight.requires_grad_()
                changed_node.layer_5.weight=torch.nn.Parameter(newweight)
     
        else:
            newweight=self.layer_out.weight.requires_grad_(False)+torch.tensor([weightchange],requires_grad=False)
            newweight.requires_grad_()
            changed_node.layer_out.weight=torch.nn.Parameter(newweight)
                
                
        return changed_node
    
    
    
    

    
    
    
    
    

    
    
class arbitraryLayerModel(nn.Module):
    def __init__(self):
        super(arbitraryLayerModel, self).__init__()
        self.layer_input=nn.Linear(5, NEURONS)
        self.layers=nn.ModuleList([nn.Linear(NEURONS, NEURONS) for i in range(TESTMODULE_LAYERS)])
        self.batchnorms=nn.ModuleList([nn.BatchNorm1d(NEURONS) for i in range(TESTMODULE_LAYERS)])
        self.layer_output=nn.Linear(NEURONS,1)
        self.relu=nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        
    def get_copy(self):
        copied_model=arbitraryLayerModel()
        copied_model.layers=nn.ModuleList()
        for i in self.layers:
            copied_model.layers.append(copy.deepcopy(i))
            
        copied_model.batchnorms=nn.ModuleList()
        for i in self.batchnorms:
            copied_model.batchnorms.append(copy.deepcopy(i))
        copied_model.layer_input=copy.deepcopy(self.layer_input)
        copied_model.layer_output=copy.deepcopy(self.layer_output)

        return copied_model
            
     
    def forward(self, inputs):
        x=self.relu(self.layer_input(inputs))
        for (layer, batchnorm) in zip(self.layers, self.batchnorms):
            x=batchnorm(x)
            x=self.relu(layer(x))
            x=self.dropout(x)
        x=self.layer_output(x)
        return(x)

    
    
    def change_node_model(self, node):
        changed_node=self.get_copy()
        
        weightchange=np.random.normal(0,1)
        weightchange=[weightchange.item()]
        zeros=[[0]]*(NEURONS-1)
        if node[0]==0:
            zeros.insert(weightchange, node[1])
            newweight=self.layer_input.weight.requires_grad_(False)+torch.tensor(zeros,requires_grad=False)
            newweight.requires_grad_()
            changed_node.layer_input.weight=torch.nn.Parameter(newweight)    
            return changed_node
            
            
        weightchange=np.random.normal(0, 1, size=NEURONS)
        weightchange=[i.item() for i in weightchange]
        zeros = [[0.0]*NEURONS]*(NEURONS-1)
        
        if node[0]==TESTMODULE_LAYERS+1:
            newweight=self.layer_output.weight.requires_grad_(False)+torch.tensor(weightchange,requires_grad=False)
            newweight.requires_grad_()
            changed_node.layer_output.weight=torch.nn.Parameter(newweight) 
            
            
        
        
        else:
            for layer, i in enumerate(changed_node.layers):
                if i==node[0]+1:
                    zeros.insert(weightchange,node[1])
                    newweight=layer.weight.requires_grad_(False)+torch.tensor(weightchange,requires_grad=False)
                    newweight.requires_grad_()
                    layer.weight=torch.nn.Parameter(newweight)
                    
        

        return changed_node

    
    
        
            


    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
def select_node():
    layer=randrange(5)
    if layer!=6:
        node=randrange(5)
    else:
        node=randrange(2)
    return [layer, node]
    
def change_node_modellinear(self, node):
        changed_node=binaryClassification()
        changed_node.layer_1 = copy.deepcopy(self.layer_1)
        changed_node.layer_2 = copy.deepcopy(self.layer_2) 
        changed_node.layer_3 = copy.deepcopy(self.layer_3)
        changed_node.layer_4 = copy.deepcopy(self.layer_4)
        changed_node.layer_5 = copy.deepcopy(self.layer_5)
        changed_node.layer_out = copy.deepcopy(self.layer_out) 
        
        weightchange=[1.0]*5

        if node[0]==0:
            self.layer_1.weight.requires_grad_(False)
            if node[1]==0:
                newweight=self.layer_1.weight+torch.tensor([weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_1.weight=torch.nn.Parameter(newweight)
                
            if node[1]==1:
                newweight=self.layer_1.weight+torch.tensor([[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)  
                newweight.requires_grad_()
                changed_node.layer_1.weight=torch.nn.Parameter(newweight)
                
            if node[1]==2:
                newweight=self.layer_1.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)          
                newweight.requires_grad_()
                changed_node.layer_1.weight=torch.nn.Parameter(newweight)
                
            if node[1]==3:
                newweight=self.layer_1.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0]],requires_grad=False)       
                newweight.requires_grad_()
                changed_node.layer_1.weight=torch.nn.Parameter(newweight)
                
            if node[1]==4:
                newweight=self.layer_1.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange],requires_grad=False)           
                newweight.requires_grad_()
                changed_node.layer_1.weight=torch.nn.Parameter(newweight)

        elif node[0]==1:
            self.layer_2.weight.requires_grad_(False)
            if node[1]==0:
                newweight=self.layer_2.weight+torch.tensor([weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_2.weight=torch.nn.Parameter(newweight)
                
            if node[1]==1:
                newweight=self.layer_2.weight+torch.tensor([[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)  
                newweight.requires_grad_()
                changed_node.layer_2.weight=torch.nn.Parameter(newweight)
                
            if node[1]==2:
                newweight=self.layer_2.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)          
                newweight.requires_grad_()
                changed_node.layer_2.weight=torch.nn.Parameter(newweight)
                
            if node[1]==3:
                newweight=self.layer_2.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0]],requires_grad=False)       
                newweight.requires_grad_()
                changed_node.layer_2.weight=torch.nn.Parameter(newweight)
                
            if node[1]==4:
                newweight=self.layer_2.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange],requires_grad=False)           
                newweight.requires_grad_()
                changed_node.layer_2.weight=torch.nn.Parameter(newweight)
   
        elif node[0]==2:
            if node[1]==0:
                newweight=self.layer_3.weight.requires_grad_(False)+torch.tensor([weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_3.weight=torch.nn.Parameter(newweight)
                
            if node[1]==1:
                newweight=self.layer_3.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)  
                newweight.requires_grad_()
                changed_node.layer_3.weight=torch.nn.Parameter(newweight)
                
            if node[1]==2:
                newweight=self.layer_3.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)          
                newweight.requires_grad_()
                changed_node.layer_3.weight=torch.nn.Parameter(newweight)
                
            if node[1]==3:
                newweight=self.layer_3.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0]],requires_grad=False)       
                newweight.requires_grad_()
                changed_node.layer_3.weight=torch.nn.Parameter(newweight)
                
            if node[1]==4:
                newweight=self.layer_3.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange],requires_grad=False)           
                newweight.requires_grad_()
                changed_node.layer_3.weight=torch.nn.Parameter(newweight)
                               
        elif node[0]==3:
            if node[1]==0:
                newweight=self.layer_4.weight.requires_grad_(False)+torch.tensor([weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_4.weight=torch.nn.Parameter(newweight)
                
            if node[1]==1:
                newweight=self.layer_4.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)  
                newweight.requires_grad_()
                changed_node.layer_4.weight=torch.nn.Parameter(newweight)
                
            if node[1]==2:
                newweight=self.layer_4.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)          
                newweight.requires_grad_()
                changed_node.layer_4.weight=torch.nn.Parameter(newweight)
                
            if node[1]==3:
                newweight=self.layer_4.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0]],requires_grad=False)       
                newweight.requires_grad_()
                changed_node.layer_4.weight=torch.nn.Parameter(newweight)
                
            if node[1]==4:
                newweight=self.layer_4.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange],requires_grad=False)           
                newweight.requires_grad_()
                changed_node.layer_4.weight=torch.nn.Parameter(newweight)
             
        elif node[0]==4:
            if node[1]==0:
                newweight=self.layer_5.weight.requires_grad_(False)+torch.tensor([weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_5.weight=torch.nn.Parameter(newweight)
                
            if node[1]==1:
                newweight=self.layer_5.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)  
                newweight.requires_grad_()
                changed_node.layer_5.weight=torch.nn.Parameter(newweight)
                
            if node[1]==2:
                newweight=self.layer_5.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)          
                newweight.requires_grad_()
                changed_node.layer_5.weight=torch.nn.Parameter(newweight)
                
            if node[1]==3:
                newweight=self.layer_5.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0]],requires_grad=False)       
                newweight.requires_grad_()
                changed_node.layer_5.weight=torch.nn.Parameter(newweight)
                
            if node[1]==4:
                newweight=self.layer_5.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange],requires_grad=False)           
                newweight.requires_grad_()
                changed_node.layer_5.weight=torch.nn.Parameter(newweight)
     
        else:
            newweight=self.layer_output.weight.requires_grad_(False)+torch.tensor([weightchange],requires_grad=False)
            newweight.requires_grad_()
            changed_node.layer_output.weight=torch.nn.Parameter(newweight)
                
                
        return changed_node
                   
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")   

    



In [ ]:
#Linear combo 5d guassian weight (cracking model has double layers)
       
#groundtruth model  
GroundTruth = binaryClassification()
firstmodel=arbitraryLayerModel()
possible_infinite_time=arbitraryLayerModel()



test_model=arbitraryLayerModel()
firstmodeltrained=test_model.get_copy()
accepted=False
current_time=0
data_list=[]


while((not accepted)):
    X, y=getdata_XY()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
    train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
    test_data = testData(torch.FloatTensor(X_test))
    train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=False)                      
    test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)
    data_list.append(X)
    current_time=current_time+trainnetwork(firstmodeltrained, train_loader)
    accepted=check_threshhold(firstmodeltrained, test_loader, y_test)
    
print("First time: ", current_time)
    
#iterates
interationcounter=1
no_increasecounter=0
time_increase=True
loss_difference_counter=0
stop_itterations=False
while(time_increase):
    node_list=[]
    time_list=[]
    model_list=[]
    for i in range(0, 10):
        node_list.append(select_node())
    for i in range(0, 10):
        current_loss=0
        previous_loss=0
        train_testmodel=test_model.get_copy()
        prospective_GroundTruth=GroundTruth.change_node_model(node_list[i])
        model_list.append(prospective_GroundTruth)
        data_counter=0
        test_time=0
        inital_data_length=len(data_list)
        accepted=False
        loss_difference_counter=0

    



        while(not accepted): 
            #use previously generated data
            if data_counter<inital_data_length:
                X=data_list[data_counter]
                y=get_yvals(prospective_GroundTruth, X)
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
                train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
                test_data = testData(torch.FloatTensor(X_test))
                train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)                      
                test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)




                test_time=test_time+trainnetwork(train_testmodel, train_loader)
                previous_loss=current_loss
                accepted, current_loss=check_threshhold(train_testmodel, test_loader, y_test)
                loss_diff=previous_loss-current_loss
                data_counter=data_counter+1
            #generate new data if needed
            else:                          
                X=get_Xvals()
                data_list.append(X)
                y=get_yvals(prospective_GroundTruth, X)



                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
                train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
                test_data = testData(torch.FloatTensor(X_test))
                train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)                      
                test_loader = DataLoader(dataset=test_data, batch_size=1)
                
                previous_loss=current_loss
                test_time=test_time+trainnetwork(train_testmodel, train_loader)
                accepted, current_loss=check_threshhold(train_testmodel, test_loader, y_test)
                loss_diff=previous_loss-current_loss

             
            
            
            #checks if loss difference is small
            if abs(loss_diff)<(1e-6):
                loss_difference_counter=loss_difference_counter+1
                if loss_difference_counter>5: 
                    print(f'Number no loss change {loss_difference_counter}: | Current time: {test_time:.3f} | Loss difference: {abs(loss_diff):.4} | Loss: {current_loss:.4}')
                
             
            
            #if to many training routines with small loss, breaks out of code
            if loss_difference_counter>=BADLOSSPASSES:
                possible_infinite_time=prospective_GroundTruth.get_copy()
                stop_itterations=True
                break

                
                
                
                
                
                

        time_list.append(test_time)
        
         
        #if to many training routines with small loss, breaks out of code
        if stop_itterations:
            print("No loss decrease for ", BADLOSSPASSES, " threshhold checks. Prospective global minimum found")
            break
            
    
    
    max_new_time=max(time_list)
    
    
    
    #prints iteration data
    print(f'Iteration {interationcounter}: | Previous time: {current_time:.3f} | Best new time: {max_new_time:.3f} | Difference: {max_new_time-current_time:.3f}')
    interationcounter=interationcounter+1
    
    #changes ground truth if better time found
    if max(time_list)>current_time:
        GroundTruth=get_new_GroundTruth(model_list, time_list, current_time)
        time_increase=True
        previous_time=current_time
        current_time=max_new_time
        no_increasecounter=0
        
    else:
        no_increasecounter=no_increasecounter+1

    #if no better ground truth found for 10 iterations, stops code                 
    if no_increasecounter==10:
        print("No increase for 10 iterations")
        time_increase=False
 
    #if to many training routines with small loss, breaks out of code
    if stop_itterations:
        break
                                
        

First time:  2.2322065830230713
Iteration 1: | Previous time: 2.232 | Best new time: 2.386 | Difference: 0.154
Iteration 2: | Previous time: 2.386 | Best new time: 2.238 | Difference: -0.148
Number no loss change 6: | Current time: 59.735 | Loss difference: 2.473e-07 | Loss: 2.3701577447354794e-05
Number no loss change 7: | Current time: 68.482 | Loss difference: 6.398e-07 | Loss: 2.1978705262881704e-05
Number no loss change 8: | Current time: 81.668 | Loss difference: 1.84e-07 | Loss: 2.1847841708222404e-05
Number no loss change 9: | Current time: 121.256 | Loss difference: 2.37e-07 | Loss: 2.527827200538013e-05
Number no loss change 10: | Current time: 127.863 | Loss difference: 6.16e-07 | Loss: 2.4285931431222707e-05
Number no loss change 11: | Current time: 148.083 | Loss difference: 1.496e-07 | Loss: 2.52058725891402e-05
Number no loss change 12: | Current time: 150.276 | Loss difference: 1.375e-07 | Loss: 2.534334453230258e-05
Number no loss change 13: | Current time: 156.946 | L

In [2]:
#add [1]*5 Not Needed
firstpass=True
time_increase=1
while(time_increase>.01):
    node_list=[]
    time_list=[]
    model_list=[]
    for i in range(0, 10):
        node_list.append(select_node())
    for i in range(0, 10):
        train_testmodel=test_model.get_copy()
        prospective_GroundTruth=GroundTruth.change_node_modellinear(node_list[i])
        model_list.append(prospective_GroundTruth)
        data_counter=0
        test_time=0
        inital_data_length=len(data_list)
        accepted=False






        while(not accepted): 
            #use previously generated data
            if data_counter<inital_data_length:
                X=data_list[data_counter]
                y=get_yvals(prospective_GroundTruth, X)
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
                train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
                test_data = testData(torch.FloatTensor(X_test))
                train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)                      
                test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)




                test_time=test_time+trainnetwork(train_testmodel, train_loader)
                accepted=check_threshhold(train_testmodel, test_loader, y_test)
                data_counter=data_counter+1
            else:                          
                X=get_Xvals()
                data_list.append(X)
                y=get_yvals(prospective_GroundTruth, X)



                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
                train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
                test_data = testData(torch.FloatTensor(X_test))
                train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)                      
                test_loader = DataLoader(dataset=test_data, batch_size=1)
                test_time=test_time+trainnetwork(train_testmodel, train_loader)
                accepted=check_threshhold(train_testmodel, test_loader, y_test)

            #print(test_time)
        time_list.append(test_time)
    
    
    print(time_list)
    max_new_time=max(time_list)
    max_index=time_list.index(max_new_time)
    GroundTruth=model_list[i]
    time_increase=max_new_time-current_time
    current_time=max_new_time

    
    
 

current loss:  [[0.00016184]]
current loss:  [[0.00111658]]
current loss:  [[0.00096108]]
current loss:  [[0.00114861]]
current loss:  [[0.00110419]]
current loss:  [[0.00043214]]
current loss:  [[0.00111681]]
current loss:  [[0.0010474]]
current loss:  [[0.00113417]]
current loss:  [[0.00110339]]
[12.976532697677612, 12.938209056854248, 14.750714302062988, 12.007935762405396, 11.025629043579102, 10.595731019973755, 12.371632814407349, 11.334189653396606, 13.096515417098999, 12.43299150466919]
current loss:  [[5.5408644e-05]]
current loss:  [[0.00071849]]
current loss:  [[0.00042576]]
current loss:  [[0.00038151]]
current loss:  [[0.00106397]]
current loss:  [[0.00112753]]
current loss:  [[0.0007163]]
current loss:  [[0.00146907]]
current loss:  [[0.00049282]]
current loss:  [[0.00100258]]
[13.589966773986816, 12.514455795288086, 12.277449607849121, 12.588509798049927, 13.095532655715942, 12.446441650390625, 12.127474069595337, 13.397139310836792, 12.944106101989746, 12.364925622940063